In [1]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.action_chains import ActionChains
from datetime import datetime, timedelta
from bs4 import BeautifulSoup
import pandas as pd
import time

list_links = []
name_links = []
raw_links = []
intreset_links = []
what_is_it = []
append_city = []
filter_name = []
run_bool = []

In [2]:
## Add here new city if not present already

city_list = [
"Munich",
"Herculaneum",
"Amalfi-Coast",
"Pompeii"]

In [56]:
## Code below only refreshes the exisitng links 
city_list = ['Munich']
path_to_file = fr'G:\.shortcut-targets-by-id\1ER8hilqZ2TuX2C34R3SMAtd1Xbk94LE2\MyOTAs\Baza Excel\Resource\GYG_links.csv'
df_cities = pd.read_csv(path_to_file)
city_list = df_cities['City'].unique().tolist()
city_list

['Amsterdam',
 'Athens',
 'Barcelona',
 'Berlin',
 'Cancun',
 'Dubai',
 'Dublin',
 'Dubrovnik',
 'Edinburgh',
 'Florence',
 'Istanbul',
 'Krakow',
 'Las-Vegas',
 'Lisbon',
 'London',
 'Madrid',
 'Milan',
 'Naples',
 'New-York-City',
 'Paris',
 'Porto',
 'Rome',
 'Palermo',
 'Venice',
 'Taormina',
 'Capri',
 'Sorrento',
 'Mount-Etna',
 'Mount-Vesuvius',
 'Herculaneum',
 'Amalfi-Coast',
 'Pompeii']

In [57]:
city_list = city_list[0:1]
city_list

['Amsterdam']

In [4]:
def extract_filter_data(soup, filter_id, filter_type, set_run):
    filter_box = soup.find('div', {'id': filter_id})
    if filter_box:
        filter_list = filter_box.find_all('li', {'class': 'search-filters-item-input'})
        for item in filter_list:
            filter_name_text = item.find('input', {'type': 'checkbox'})['name'].split('-')[-1]
            # Convert filter names to a consistent format if necessary
            if filter_name_text == 'activityType':
                filter_name_text = 'activity_type'
            elif filter_name_text == 'poiTypes':
                filter_name_text = 'poi_types'
            elif filter_name_text == 'transportationTypes':
                filter_name_text = 'transportation_types'
            elif filter_name_text == 'locations':
                filter_name_text = 'lc'
            # Construct and append data
            list_links.append(base_url + f'&{filter_name_text}=' + item.find('label')['for'].split('x-')[-1])
            name_links.append(item.find('label').text.strip())
            raw_links.append(item.find('label')['for'].split('x-')[-1])
            what_is_it.append(filter_type)
            append_city.append(new_city)
            filter_name.append(filter_name_text)
            if set_run:
                run_bool.append(1)
            else:
                run_bool.append(0)
                

# Setup WebDriver
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()))
driver.maximize_window()

list_links, name_links, raw_links, what_is_it, append_city, filter_namem, run_bool = [], [], [], [], [], [], []

for new_city in city_list:
    base_url = f"https://www.getyourguide.com/s?q={new_city}"
    driver.get(base_url)
    WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.CLASS_NAME, 'c-chip')))
    city_input = driver.find_elements(By.CLASS_NAME, 'c-chip')
    for city in city_input:
        if city.text in ["Price", "Filters"]:
            city.click()

    WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.TAG_NAME, 'body')))
    html = driver.page_source
    soup = BeautifulSoup(html, 'html.parser')

    # Append the 'Global' link data for each city before extracting specific filter data
    list_links.append(base_url)
    name_links.append('Global')
    raw_links.append('Global')
    what_is_it.append('Category')
    append_city.append(new_city)
    filter_name.append('global')
    run_bool.append(1)

    # Process each filter type
    extract_filter_data(soup, 'accordion-panel-activityType', 'Category', True)
    extract_filter_data(soup, 'accordion-panel-interests', 'Intrests', False)
    extract_filter_data(soup, 'accordion-panel-locations', 'Destination', True)

# Construct the DataFrame
df_links = pd.DataFrame({
    'URL': list_links,
    'Category': name_links,
    'RawCategory': raw_links,
    'WhatIsIt': what_is_it,
    'City': append_city,
    'FilterName': filter_name,
    'Run': run_bool
})

df_links.drop_duplicates(subset=('URL'), inplace=True)

# Optional: Cleanup and transformations
# Example: df_links['MatchCategory'] = df_links['RawCategory'].apply(lambda x: x.split('/')[2])

display(df_links)
driver.quit()

,URL,Category,RawCategory,WhatIsIt,City,FilterName,Run
0,https://www.getyourguide.com/s?q=Munich,Global,Global,Category,Munich,global,1
1,https://www.getyourguide.com/s?q=Munich&activi...,Entry tickets,entryTicket,Category,Munich,activity_type,1
2,https://www.getyourguide.com/s?q=Munich&activi...,Guided tours,guidedTour,Category,Munich,activity_type,1
3,https://www.getyourguide.com/s?q=Munich&activi...,Other experiences,other,Category,Munich,activity_type,1
4,https://www.getyourguide.com/s?q=Munich&activi...,Private tours,privateTour,Category,Munich,activity_type,1
5,https://www.getyourguide.com/s?q=Munich&activi...,City cards,cityCard,Category,Munich,activity_type,1
6,https://www.getyourguide.com/s?q=Munich&activi...,Day trips,dayTrip,Category,Munich,activity_type,1
7,https://www.getyourguide.com/s?q=Munich&activi...,Transfers,transfer,Category,Munich,activity_type,1
8,https://www.getyourguide.com/s?q=Munich&activi...,Water activities,waterActivity,Category,Munich,activity_type,1
9,https://www.getyourguide.com/s?q=Munich&activi...,Hop-on hop-off tours,hopOnHopOff,Category,Munich,activity_type,1


In [45]:
# df_links.to_csv(r'C:\Users\Wojciech\Documents\Python\MyOTAs\resource\2023-06-20_GYG_links.csv')
date_today = datetime.today().strftime("%Y-%m-%d")
df_links.to_csv(fr'G:\.shortcut-targets-by-id\1ER8hilqZ2TuX2C34R3SMAtd1Xbk94LE2\MyOTAs\Baza Excel\Resource\GYG\{date_today}_GYG_links.csv')